In [1]:
import sys
sys.path.append("../../pybdp")
from src import pybdp
#import pybdp
from pprint import pprint

# Start with an empty project
project = pybdp.create_empty_project()

project.add_space(id = "X",
                  name = "X",
                  description = "Input vector")

project.add_space(id = "x_i",
                  name = "x_i",
                  description = "Individual token")

project.add_space(id = "h_i",
                  name = "h_i",
                  description = "Hidden state")

project.add_space(id = "c",
                  name = "c",
                  description = "Context vector")

project.add_block(id="RNN Hidden Layer",
                  name="RNN Hidden Layer",
                  description="A hidden layer of an RNN",
                  domain=["x_i", "h_i"],
                  codomain=["h_i"],)

project.add_block(id="Input Layer Block Length 3",
                  name="Input Layer Block Length 3",
                  description="The input layer which transforms the input vector into a sequence of tokens",
                  domain=["X"],
                  codomain=["x_i", "x_i", "x_i"],)

project.add_processor(id="Input Layer Length 3",
                  description="The input layer which transforms the input vector into a sequence of tokens",
                  parent_id="Input Layer Block Length 3",)


project.add_processor(id="RNN Encoder Cell 1", parent_id="RNN Hidden Layer")
project.add_processor(id="RNN Encoder Cell 2", parent_id="RNN Hidden Layer")
project.add_processor(id="RNN Encoder Cell 3", parent_id="RNN Hidden Layer")

In [2]:
pprint(project.processors_map["RNN Encoder Cell 1"].find_potential_wires(project.processors_map["RNN Encoder Cell 2"]))
pprint(project.processors_map["RNN Encoder Cell 2"].find_potential_wires(project.processors_map["RNN Encoder Cell 3"]))

{'Ports': [{'Parent': 'h_i',
            'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'},
            'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 1'}}],
 'Terminals': [{'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 1'},
                'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 2'}}]}
{'Ports': [{'Parent': 'h_i',
            'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 3'},
            'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 2'}}],
 'Terminals': [{'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'},
                'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 3'}}]}


In [3]:
project.add_wires([{'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 1'},
                'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 2'}},
                {'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'},
                'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 3'}}], auto_increment=True)


project.add_wires([{'Parent': 'x_i',
                'Source': {'Index': 0, 'Processor': 'Input Layer Length 3'},
                'Target': {'Index': 0, 'Processor': 'RNN Encoder Cell 1'}},
                {'Parent': 'x_i',
                'Source': {'Index': 1, 'Processor': 'Input Layer Length 3'},
                'Target': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'}},
                {'Parent': 'x_i',
                'Source': {'Index': 2, 'Processor': 'Input Layer Length 3'},
                'Target': {'Index': 0, 'Processor': 'RNN Encoder Cell 3'}}], auto_increment=True)



In [4]:
project.add_system(id="RNN Encoder System Length 3",
                   processors=['RNN Encoder Cell 1', 'RNN Encoder Cell 2', 'RNN Encoder Cell 3', 'Input Layer Length 3'],
                   wires=["W1", "W2", "W3", "W4", "W5"],
                     description="The RNN encoder system",)

project.systems_map["RNN Encoder System Length 3"].display_mermaid_graphic()

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GS0[RNN Encoder System Length 3]
subgraph G0[RNN Encoder Cell 1 - RNN Hidden Layer Block]
direction LR
X0[RNN Encoder Cell 1]
subgraph G0P[Ports]
direction TB
XX0P0[x_i]
XX0P1[h_i]
end
XX0P0[x_i] o--o X0
XX0P1[h_i] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[h_i]
end
X0 o--o XX0T0[h_i]
end
subgraph G1[RNN Encoder Cell 2 - RNN Hidden Layer Block]
direction LR
X1[RNN Encoder Cell 2]
subgraph G1P[Ports]
direction TB
XX1P0[x_i]
XX1P1[h_i]
end
XX1P0[x_i] o--o X1
XX1P1[h_i] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[h_i]
end
X1 o--o XX1T0[h_i]
end
subgraph G2[RNN Encoder Cell 3 - RNN Hidden Layer Block]
direction LR
X2[RNN Encoder Cell 3]
subgraph G2P[Ports]
direction TB
XX2P0[x_i]
XX2P1[h_i]
end
XX2P0[x_i] o--o X2
XX2P1[h_i] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[h_i]
end
X2 o--o XX2T0[h_i]
end
subgraph G3[Input Layer Length 3 - Input Layer Block Length 3 Block]
direction LR
X3[Input Layer Length 3]
subgraph G3P[Ports]
direction TB
XX3P0[X]
end
XX3P0[X] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[x_i]
XX3T1[x_i]
XX3T2[x_i]
end
X3 o--o XX3T0[x_i]
X3 o--o XX3T1[x_i]
X3 o--o XX3T2[x_i]
end
XX0T0[h_i] ---> XX1P1[h_i]
XX1T0[h_i] ---> XX2P1[h_i]
XX3T0[x_i] ---> XX0P0[x_i]
XX3T1[x_i] ---> XX1P0[x_i]
XX3T2[x_i] ---> XX2P0[x_i]
end

```

In [5]:
project.systems_map["RNN Encoder System Length 3"].make_processor_lazy(terminals=[["RNN Encoder Cell 1", 0, "h_i"],
                                                                              ["RNN Encoder Cell 2", 0, "h_i"],
                                                                              ["RNN Encoder Cell 3", 0, "h_i"]],
                                                                    block_id="RNN Encoder Block Length 3",
                                                                    block_name="RNN Encoder Block Length 3",
                                                                    processor_id="RNN Encoder Length 3",
                                                                    processor_name="RNN Encoder Length 3",
                                                                    block_description="An RNN Encoder Block with length 3",
                                                                    processor_description="An RNN Encoder with length 3",
                                                                    add=True,
                                                                    project=project,)
project.processors_map["RNN Encoder Length 3"].display_mermaid_graphic(composite=True)

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[RNN Encoder Length 3 - RNN Encoder Block Length 3 Block]
direction LR
subgraph GS0[RNN Encoder System Length 3]
subgraph G1[RNN Encoder Cell 1 - RNN Hidden Layer Block]
direction LR
X1[RNN Encoder Cell 1]
subgraph G1P[Ports]
direction TB
XX1P0[x_i]
XX1P1[h_i]
end
XX1P0[x_i] o--o X1
XX1P1[h_i] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[h_i]
end
X1 o--o XX1T0[h_i]
end
subgraph G2[RNN Encoder Cell 2 - RNN Hidden Layer Block]
direction LR
X2[RNN Encoder Cell 2]
subgraph G2P[Ports]
direction TB
XX2P0[x_i]
XX2P1[h_i]
end
XX2P0[x_i] o--o X2
XX2P1[h_i] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[h_i]
end
X2 o--o XX2T0[h_i]
end
subgraph G3[RNN Encoder Cell 3 - RNN Hidden Layer Block]
direction LR
X3[RNN Encoder Cell 3]
subgraph G3P[Ports]
direction TB
XX3P0[x_i]
XX3P1[h_i]
end
XX3P0[x_i] o--o X3
XX3P1[h_i] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[h_i]
end
X3 o--o XX3T0[h_i]
end
subgraph G4[Input Layer Length 3 - Input Layer Block Length 3 Block]
direction LR
X4[Input Layer Length 3]
subgraph G4P[Ports]
direction TB
XX4P0[X]
end
XX4P0[X] o--o X4
subgraph G4T[Terminals]
direction TB
XX4T0[x_i]
XX4T1[x_i]
XX4T2[x_i]
end
X4 o--o XX4T0[x_i]
X4 o--o XX4T1[x_i]
X4 o--o XX4T2[x_i]
end
XX1T0[h_i] ---> XX2P1[h_i]
XX2T0[h_i] ---> XX3P1[h_i]
XX4T0[x_i] ---> XX1P0[x_i]
XX4T1[x_i] ---> XX2P0[x_i]
XX4T2[x_i] ---> XX3P0[x_i]
end
subgraph GC0P[Ports]
direction TB
X1P0[h_i]
X1P1[X]
end
X1P0[h_i] --> XX1P1[h_i]
X1P1[X] --> XX4P0[X]
subgraph GC0T[Terminals]
direction TB
X1T0[h_i]
X1T1[h_i]
X1T2[h_i]
end
XX1T0[h_i] --> X1T0[h_i]
XX2T0[h_i] --> X1T1[h_i]
XX3T0[h_i] --> X1T2[h_i]
end

```